# Advent of Code 2022

In [1]:
YEAR=2022

# Helpers


These helper functions are useful to answer the questions. These (and using a Jupyter notebook in the first place) are inspired by Peter Norvig

## Robustness

In [2]:
# Allows for refactoring after having determined the answer.
def assert_eq(expected, got, show, msg=None):
    if expected == got:
        return expected
    else:
        if msg:
            print("Extra info:", msg)
        raise Exception(f"Expected '{expected}', but got '{got}'")

def assert_example(expected, got, msg=None):
    assert_eq(expected, got, show=False, msg=msg)

def assert_answer(name, expected, got):
    print("Answer for", name, "is", assert_eq(expected, got, show=True))

## Input processing

In [3]:
def read_input(name, strip=True):
    with open(f"inputs/{YEAR}/{name}") as f:
        data = f.read()
        if strip:
            data = data.strip()
        return data
    
# Parse a input file, where each item (line by default) is pared by function passed to `line_parser` 
def parse_input(name, parser=str, sep="\n", preview_len=10,preview=True):
    items = tmap(parser, read_input(name).split(sep))
    if preview:
        print(f"Parsing '{name}'. Total items: {len(items)}")
        print(f"First {preview_len} items:")
        for preview in items[0:preview_len]:
          print(" ", preview)
    return items

# Splits an item on a separator, and then either apply the one function, or list of functions to the parts.
def parse_split(sep=" ", fns=str):
    def inner(x):
        parts = x.split(sep)
        if isinstance(fns, list):
            return [fn(part) for (fn,part) in zip(fns, parts)]
        else:
            return [fns(part) for part in parts]
    return inner

## Strict functions

In [4]:
# By default, `map` and `filter` are lazy. Not always practical in a notebook where you might want to inspect the data :)
def tmap(fn, *args):
    return tuple(map(fn, *args))
def tfilter(fn, *args):
    return tuple(filter(fn, *args))

## Data analysis

In [5]:
import itertools
from functools import reduce

def count_true(data):
    return sum([1 for item in data if item])

def count_pred_true(pred, data):
    return sum([1 for item in data if pred(item)])

# Return a sliding window (filled) of size `ws`
def windowed(data, ws):
    return [data[i:i+ws] for i in range(0, len(data)-ws+1)]

# Flatten a list of list one level, into one list.
def flatten(list_of_lists):
    return list(itertools.chain(*list_of_lists))

# Day 1: Calorie Counting

## Input
* [Example input](inputs/2022/01-example)
* [Personal input](inputs/2022/01)

The input format consists of paragraphs of numbers (e.g. separated by `\n\n`).
We need to find the higest (puzzle 1) and 3 highest summations of the paragraphs.


## Puzzle 1

In [6]:
# We parse the input by splitting into paragraphs, mapping the paragraphs into lines, converting the lines into
def parse_input01(name):
    data = parse_input(name, sep="\n\n", parser=parse_split("\n", int))
    return sorted([sum(elf_calories) for elf_calories in data], reverse=True)

In [7]:
example_input01 = parse_input01("01-example")

def puzzle_01a(elves):
    return elves[0]

assert_example(24000, puzzle_01a(example_input01))

Parsing '01-example'. Total items: 5
First 10 items:
  [1000, 2000, 3000]
  [4000]
  [5000, 6000]
  [7000, 8000, 9000]
  [10000]


In [8]:
input01 = parse_input01("01")

assert_answer("1a", 67658, puzzle_01a(input01))

Parsing '01'. Total items: 241
First 10 items:
  [17998, 7761]
  [5628, 1490, 4416, 2606, 2828, 4615, 3206, 7218, 4793, 5199, 2129]
  [6451, 5761, 5083, 4234, 3772, 6652, 6856, 4224, 3168, 2459, 3736, 3020, 1545]
  [3562, 6789, 4611, 1955, 5887, 7005, 5044, 6414, 1524, 3016, 3960, 1040]
  [5057, 6712, 2704, 7012, 5828, 4114, 3850, 5661, 2520, 4217, 2117, 1732]
  [10514]
  [25104, 24972, 13901]
  [28648, 33075]
  [5272, 3309, 1538, 4691, 5707, 3432, 5693, 2276, 1805, 3794, 3362, 1104, 1445, 1771, 4570]
  [56986]
Answer for 1a is 67658


## Puzzle 2

In [9]:
def puzzle_01b(elves):
    return sum(elves[0:3])

assert_answer("1b", 200158, puzzle_01b(input01))


Answer for 1b is 200158


# Day 2: Rock Paper Scissors

## Input
* [Example input](inputs/2022/02-example)
* [Personal input](inputs/2022/01)

The input format consists of a list of games, where 

-  the first column has values `A`, `B` or `C` maps to the choice of the elf for Rock, Paper or Scissors)
-  the second column has values `X`, `Y` or `Z` maps to either for puzzle one, to for Rock, Paper or Scissors, or if we should lose, play a draw, or win a game.

For the first puzzle, we evaluate a score for each game, and sum it.
For the second puzzle, we need to determine our move to match the desired win result, and then apply the scoring from the 1st puzzle.


## Puzzle 1

In [10]:
ord("a") - ord("a")+1 # use ASCII vars for parsing

1

In [11]:
ROCK = 1 
PAPER = 2
SCISSORS = 3

def puzzle02a_eval_game(game):
    (elf, me)=game
    score = me
    if elf == me:
        score += 3
    elif (me == ROCK and elf == SCISSORS) or (me == SCISSORS and elf == PAPER) or (me == PAPER and elf == ROCK):
        score += 6
    return score

assert_example(ROCK, puzzle02a_eval_game([PAPER, ROCK]))
assert_example(PAPER+6, puzzle02a_eval_game([ROCK, PAPER]))
assert_example(PAPER+3, puzzle02a_eval_game([PAPER, PAPER]))

In [12]:
def parse_puzzle2(name):
    return parse_input(name, parse_split(fns=[lambda x: ord(x) - ord("A")+1, lambda y: ord(y) - ord("X")+1]))

example_input02 = parse_puzzle2("02-example")

def puzzle_02a(games):
    return sum(tmap(puzzle02a_eval_game, games))

assert_example(15, puzzle_02a(example_input02))

input02 = parse_puzzle2("02")

assert_answer("2a", 14531, puzzle_02a(input02))


Parsing '02-example'. Total items: 3
First 10 items:
  [1, 2]
  [2, 1]
  [3, 3]
Parsing '02'. Total items: 2500
First 10 items:
  [1, 2]
  [3, 2]
  [3, 1]
  [1, 2]
  [3, 1]
  [3, 2]
  [2, 3]
  [1, 2]
  [1, 2]
  [3, 3]
Answer for 2a is 14531


## Puzzle 2

In [13]:
LOSE = 1
DRAW = 2
WIN = 3

def puzzle02b_eval_game(game):
    # I thought about encoding this with a modulo operator, but that gave me a headache :)
    (elf, must_score) = game
    if must_score == DRAW:
        me = elf
    elif must_score == LOSE:
        if elf == ROCK:     me = SCISSORS
        if elf == PAPER:    me = ROCK
        if elf == SCISSORS: me = PAPER
    elif must_score == WIN:
        if elf == ROCK:     me = PAPER
        if elf == PAPER:    me = SCISSORS
        if elf == SCISSORS: me = ROCK
    return puzzle02a_eval_game([elf, me])

assert_example(ROCK, puzzle02b_eval_game([PAPER, LOSE]))
assert_example(PAPER, puzzle02b_eval_game([SCISSORS, LOSE]))
assert_example(SCISSORS, puzzle02b_eval_game([ROCK, LOSE]))

def puzzle_02b(games):
    return sum(tmap(puzzle02b_eval_game, games))

assert_example(12, puzzle_02b(example_input02))

assert_answer("2a", 11258, puzzle_02b(input02))


Answer for 2a is 11258


# Day 3: Rucksack Reorganization

In [14]:
example_input03 = parse_input("03-example", parser=lambda line: (line[0:len(line)//2], line[len(line)//2:]))

def common_letter(rucksack):
    a,b = tmap(set, rucksack)
    return tuple(a.intersection(b))[0]

def priority(letter):
    if letter >= 'A' and letter <= 'Z': return ord(letter) - ord("A") + 27
    if letter >= 'a' and letter <= 'z': return ord(letter) - ord("a") + 1

def puzzle_03a(rucksacks):
    common_letters = tmap(common_letter, rucksacks)
    priorities = tmap(priority, common_letters)
    return sum(priorities)
    
assert_example(157, puzzle_03a(example_input03))

Parsing '03-example'. Total items: 6
First 10 items:
  ('vJrwpWtwJgWr', 'hcsFMMfFFhFp')
  ('jqHRNqRjqzjGDLGL', 'rsFMfFZSrLrFZsSL')
  ('PmmdzqPrV', 'vPwwTWBwg')
  ('wMqvLMZHhHMvwLH', 'jbvcjnnSBnvTQFn')
  ('ttgJtRGJ', 'QctTZtZT')
  ('CrZsJsPPZsGz', 'wwsLwLmpwMDw')


In [15]:
input03a = parse_input("03", parser=lambda line: (line[0:len(line)//2], line[len(line)//2:]))
assert_answer("3a", 7597, puzzle_03a(input03a))

Parsing '03'. Total items: 300
First 10 items:
  ('zBBtHnnHtwwH', 'plmlRlzPLCpp')
  ('vvhJccJFGFcNsdNNJbhJsJ', 'QplQMRLQMlfdfTPCLfQQCT')
  ('GPhjcjhZ', 'DjWtnSVH')
  ('BNhHVhrGNVTbDHdD', 'JdJRPJdSQQSJwPjR')
  ('lvtsfbsqz', 'wSnJcvjSm')
  ('MftttFLftZMLgtgMbltMqZzb', 'DNrTpVGhNWrDTrpTGNpZGZhD')
  ('VSSHcTgTtTdt', 'llZlzmmbljTn')
  ('RqMqsFfQLLFLQFMMfRLPZLvP', 'pCfWrbpmCbjCnfjlWmnrmmnm')
  ('hqRDqPDRsqN', 'HwtHSNBZtJd')
  ('tNFDpDFrtdjfmjjjFmFFd', 'ScpZhZScTJgpHccHhMJgS')
Answer for 3a is 7597


In [16]:
def parse_03b(name):
    lines = read_input(name).split("\n")
    while len(lines) > 0:
        yield lines[:3]
        lines = lines[3:]

def common_letter(rucksacks):
    return list(reduce(lambda a,b: a.intersection(b), tmap(set, rucksacks)))[0]
        
def puzzle_03b(rucksacks):
    common_letters = tmap(common_letter, rucksacks)
    priorities = tmap(priority, common_letters)
    return sum(priorities)        
            
example_input03b = list(parse_03b("03-example"))
assert_example(70, puzzle_03b(example_input03b))

input03b = list(parse_03b("03"))
assert_answer("3b", 2607, puzzle_03b(input03b))

Answer for 3b is 2607


# Day 4: Camp Cleanup

In [17]:
def parse_04(line):
    ranges = line.split(",")
    return tmap(lambda r: tmap(int, r.split("-")), ranges)
    
example_input04 = parse_input("04-example", parser=parse_04)


Parsing '04-example'. Total items: 6
First 10 items:
  ((2, 4), (6, 8))
  ((2, 3), (4, 5))
  ((5, 7), (7, 9))
  ((2, 8), (3, 7))
  ((6, 6), (4, 6))
  ((2, 6), (4, 8))


In [18]:
def range_contained(a,b):
    return (a[0] >= b[0] and a[1] <= b[1]) or(b[0] >= a[0] and b[1] <= a[1])

def count_overlapping_ranges(elf_pairs):
    return count_true([range_contained(*pair) for pair in elf_pairs])

assert_example(2, count_overlapping_ranges(example_input04))

In [19]:
input04 = parse_input("04", parser=parse_04)
assert_answer("4a", 511, count_overlapping_ranges(input04))

Parsing '04'. Total items: 1000
First 10 items:
  ((21, 81), (20, 96))
  ((14, 80), (14, 79))
  ((87, 89), (7, 88))
  ((82, 93), (44, 82))
  ((61, 93), (62, 86))
  ((80, 94), (4, 80))
  ((66, 90), (89, 91))
  ((9, 70), (10, 10))
  ((95, 97), (26, 96))
  ((82, 87), (61, 88))
Answer for 4a is 511


In [20]:
def range_overlap(a,b):
    def inner(x,y):
        return x[0] >= y[0] and x[0] <= y[1]    
    return inner(a,b) or inner(b,a)

for nope in example_input04[0:2]:
    assert_example(False, range_overlap(*nope), msg=nope)

for yep in example_input04[2:]:
    assert_example(True, range_overlap(*yep), msg=yep)

def count_overlapping_ranges2(elf_pairs):
    return count_true([range_overlap(*pair) for pair in elf_pairs])
            

assert_example(4, count_overlapping_ranges2(example_input04))

In [21]:
assert_answer("4b", 821, count_overlapping_ranges2(input04))

Answer for 4b is 821


# Day 5: Supply Stacks

In [22]:
class Column:
    def __init__(self, values):
        self.values = values
    def take(self):
        return self.values.pop()
    def add(self, item):
        self.values.append(item)
    def peek(self):
        if len(self.values) > 0:
            return self.values[-1]
        else:
            return ""
    def take2(self, nr):
        new = self.values[0:len(self.values)-nr]
        taken = self.values[len(self.values)-nr:]
        self.values = new
        return taken
    def add2(self, items):
        self.values += items
        
        
def parse_stack(stack):
    lines = stack.split("\n")
    nr_columns=int(lines.pop().strip().split(" ")[-1])
    columns=[]
    stack_height = len(lines)
    for column_idx in range(0, nr_columns):
        column = []
        for row_idx in range(0, stack_height):
            c = lines[row_idx][4*column_idx+1]
            if c != " ":
                column.append(c)
        col = list(column)
        col.reverse()
        columns.append(Column(col))
    return columns

def viz_stack(stack):
    for idx, column in enumerate(stack):
        print(idx+1, "".join([" " if x is None else x for x in column.values]))

def parse_instructions(txt_instructions):
    instructions = []
    for line in txt_instructions.strip().split("\n"):
        _mv, mv_count, _from, from_stack, _to, to_stack = line.split(" ")
        instructions.append(tmap(int, (from_stack, to_stack, mv_count)))
    return instructions

def run_instruction(stack, instruction, nr2):
    stack_from, stack_to, take = instruction
    if not nr2:
        for i in range(0, take):
            taken = stack[stack_from-1].take()
            stack[stack_to-1].add(taken)
    else:
        taken = stack[stack_from-1].take2(take)
        stack[stack_to-1].add2(taken)
    return stack

    
def run05a(name, debug=False, nr2=False):
    stack, instructions = read_input(name, strip=False).split("\n\n")
    stack, instructions = parse_stack(stack), parse_instructions(instructions)
    
    if debug:
        print("initial stack:"); viz_stack(stack); print();
    
    for instruction in instructions:
        if debug:
            print("Instruction", instruction)
        stack = run_instruction(stack, instruction, nr2)
        if debug:
            print("stack after:"); viz_stack(stack); print();
    
    return "".join([column.peek() for column in stack])
        
assert_example("CMZ", run05a("05-example"))
assert_answer("5a", "TQRFCBSJJ", run05a("05"))
assert_example("MCD", run05a("05-example", nr2=True))
assert_example("RMHFJNVFP", run05a("05", nr2=True))

Answer for 5a is TQRFCBSJJ


# Day 6: Tuning Trouble

In [40]:
def packet_marker_pos(stream, marker_len):
    idx=marker_len
    while True:
        last=stream[(idx-marker_len):idx]
        if len(set(last)) == len(last):
            return idx
        else:
            idx+=1
               
examples = [
    [7, "mjqjpqmgbljsphdztnvjfqwrcgsmlb"],
    [5, "bvwbjplbgvbhsrlpgdmjqwftvncz"],
    [6, "nppdvjthqldpwncqszvftbrmjlhg"],
    [10, "nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg"],
    [11, "zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw"]
]
for example in examples:    
    assert_example(example[0], packet_marker_pos(example[1],4))

input_06=read_input("06")

assert_answer("6a", 1855, packet_marker_pos(input_06,4))

examples = [
    [19, "mjqjpqmgbljsphdztnvjfqwrcgsmlb"],
    [23, "bvwbjplbgvbhsrlpgdmjqwftvncz"],
    [23, "nppdvjthqldpwncqszvftbrmjlhg"],
    [29, "nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg"],
    [26, "zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw"]
]
for example in examples:    
    assert_example(example[0], packet_marker_pos(example[1],14))
    
assert_answer("6b", 3256, packet_marker_pos(input_06,14))

Answer for 6a is 1855
Answer for 6b is 3256


# Day 7: No Space Left On Device

In [74]:
def parse_07(name):
    lines = list(parse_input(name, parser=parse_split()))
    path = []
    files = []
    dirs = ["/"]
    while len(lines) > 0:
        line = lines.pop(0)
        if line[0] == "$":
            if line[1] == "cd":
                to = line[2]
                if to == "..":
                    path.pop()
                else:
                    path.append(to)
        else:
            data, name = line[0], line[1]
            full_path = "/" + "/".join(path[1:] + [name])
            if data == "dir":
                dirs.append(full_path)
            else:
                size = int(data)
                files.append((full_path, size))
    return (dirs, files)


def sum_dirs(dirs, files):
    sums=[]
    for d in dirs:
        total = 0
        for file, size in files:
            if file.startswith(d):
                total += size
        sums.append((d, total))
    return sums
        
            
example_input07=parse_07("07-example")
example_input07

example_sums = sum_dirs(*example_input07)
print(example_sums)

assert_example(48381165, example_sums[0][1])

def pluck_idx(idx, data):
    return [d[idx] for d in data]

def puzzle_7a(sums):
    return sum(pluck_idx(1, filter(lambda d: d[1] < 100000, sums)))
assert_example(95437, puzzle_7a(example_sums))

input07=parse_07("07")
sums = sum_dirs(*input07)
assert_answer("7a", 1118405, puzzle_7a(sums))

Parsing '07-example'. Total items: 23
First 10 items:
  ['$', 'cd', '/']
  ['$', 'ls']
  ['dir', 'a']
  ['14848514', 'b.txt']
  ['8504156', 'c.dat']
  ['dir', 'd']
  ['$', 'cd', 'a']
  ['$', 'ls']
  ['dir', 'e']
  ['29116', 'f']
[('/', 48381165), ('/a', 94853), ('/d', 24933642), ('/a/e', 584)]
Parsing '07'. Total items: 1001
First 10 items:
  ['$', 'cd', '/']
  ['$', 'ls']
  ['dir', 'bqm']
  ['dir', 'ctztn']
  ['dir', 'dbclg']
  ['dir', 'fhndmnt']
  ['dir', 'gczqbh']
  ['276177', 'hvbf.lvm']
  ['dir', 'lnsgbqp']
  ['dir', 'pblb']
Answer for 7a is 1118405


In [83]:
TOTAL_SIZE=70000000
NEEDED=30000000

def find_dirs_to_delete(dirs, files):
    sums = sum_dirs(dirs, files)
    size_of_root = sums[0][1]
    current_free = TOTAL_SIZE - size_of_root
    need_to_find = NEEDED - current_free
    candidates = list(filter(lambda d: d[1] > need_to_find, sums))
    sorted(candidates, key=lambda d: d[0])
    return candidates[-1]
assert_example(('/d', 24933642), find_dirs_to_delete(*example_input07))
assert_answer("7b", ('/wjgh/cqh/blvcjl', 12545514), find_dirs_to_delete(*input07))
    

Answer for 7b is ('/wjgh/cqh/blvcjl', 12545514)
